# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='D:\PythonProjects\Spyder\chromedriver')
browser.get("https://www.ettoday.net/news/news-list.htm")

# 每個兩秒鐘自動往下滑 ----------------------------------------------------------
import time

todayStr = str(datetime.date.today())
cnt=todayStr.find('-')
if cnt>0:
    todayStr=todayStr.replace('-', '/', 2)
    
# 每個兩秒鐘自動往下滑 --------------------------------------------------
forBrk=0
for i in range(1000):
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, 10000);")
    # 取得資料，丟到 BeautifulSoup 解析 ------------------------------------
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")

    for d in soup.find(class_="part_list_2").find_all('h3'):
        tmStr=d.find(class_="date").text
        titleDate=tmStr[0:10]
        if todayStr==titleDate:
            print(d.find(class_="date").text, d.find_all('a')[-1].text)
        else:
            forBrk=9
            break
    if forBrk>=1:
        break

# 關閉瀏覽器 -------------------------------------------------------
browser.quit();


'\nYour Code\n'

## 2. 取出現在時間兩小時內的新聞

In [2]:
# 最近兩HR.新聞 ------------------------------------------------------------
import datetime

tmNow = datetime.datetime.now().replace(microsecond=0)
nowSecs=tmNow.timestamp()
hrSecs=60*60
brkSecs=2*hrSecs

url = 'https://www.ettoday.net/news/news-list.htm'
r = requests.get(url)
soup = BeautifulSoup(r.text, "html5lib")

for d in soup.find(class_="part_list_2").find_all('h3'):
    tmStr=d.find(class_="date").text
    datetimeTitle = datetime.datetime.strptime(tmStr, '%Y/%m/%d %H:%M')
    titleSecs=datetimeTitle.timestamp()
    tmDiff=nowSecs-titleSecs
    if tmDiff>= brkSecs:
        break
    print(d.find(class_="date").text, d.find_all('a')[-1].text)


'\nYour Code\n'

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import datetime

# time to sec ------------------------------------------------------
def t2s(t, cnt):
    if(cnt>1):
        h,m,s = t.strip().split(":")
    else:
        h,m = t.strip().split(":")
        s='0'
    return int(h) * 3600 + int(m) * 60 + int(s)

# 打開瀏覽器 -------------------------------------------------------
browser = webdriver.Chrome(executable_path='D:\PythonProjects\Spyder\chromedriver')
browser.get("https://www.ettoday.net/news/news-list.htm")

tmNow = datetime.datetime.now().replace(microsecond=0)
ddelay = datetime.timedelta(days=3)
reqTm=tmNow-ddelay
strYr=str(reqTm.year)
strMn=str(reqTm.month)
strDy=str(reqTm.day)

selectYear = Select(browser.find_element_by_id("selY"))
selectYear.select_by_value(strYr)
selectMon = Select(browser.find_element_by_id("selM"))
selectMon.select_by_value(strMn)
selectDay = Select(browser.find_element_by_id("selD"))
selectDay.select_by_value(strDy)

browser.find_element_by_id('button').click()

# 每個兩秒鐘自動往下滑 --------------------------------------------------
strtSecs=t2s('15:00:00', 2)
endSecs=t2s('17:00:00', 2)

forBrk=0
for i in range(200):
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, 10000);")
    # 取得資料，丟到 BeautifulSoup 解析 ------------------------------------
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")

    for d in soup.find(class_="part_list_2").find_all('h3'):
        tmStr=d.find(class_="date").text
        titleSecs=t2s(tmStr[11:16], 1)
        if (titleSecs<=endSecs) and (titleSecs>=strtSecs):
            print(d.find(class_="date").text, d.find_all('a')[-1].text)
        if titleSecs<strtSecs:
            forBrk=9
            break
    if forBrk>=1:
        break

# 關閉瀏覽器 -------------------------------------------------------
browser.quit();


'\nYour Code\n'